# Sentiment classification

Below are imports and helper functions from previous classification labs. The task for this week is to build your own classifier for predicting the sentiment of Tweets. Tweets are provided from [SemEval-2016 Task 4 (Subtask A)](http://alt.qcri.org/semeval2016/task4/).

In [1]:
import ftfy
import nltk
import json

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Binarizer

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import re

from collections import Counter
from os import listdir, makedirs
from os.path import isfile, join, splitext, split

For POS tagger, incase these haven't been previously downloaded.

In [2]:
nltk.download('punkt')
nltk.download('maxent_treebank_pos_tagger')

[nltk_data] Downloading package punkt to /home/jay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/jay/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!


True

A couple of methods for showing classifier results (from 1st classification lab):

In [3]:
def print_cv_scores_summary(name, scores):
    print("{}: mean = {:.2f}%, sd = {:.2f}%, min = {:.2f}, max = {:.2f}".format(name, scores.mean()*100, scores.std()*100, scores.min()*100, scores.max()*100))
    
def confusion_matrix_heatmap(cm, index):
    cmdf = pd.DataFrame(cm, index = index, columns=index)
    dims = (10, 10)
    fig, ax = plt.subplots(figsize=dims)
    sns.heatmap(cmdf, annot=True, cmap="coolwarm", center=0)
    ax.set_ylabel('Actual')    
    ax.set_xlabel('Predicted')
    
def confusion_matrix_percent_heatmap(cm, index):
    cmdf = pd.DataFrame(cm, index = index, columns=index)
    percents = cmdf.div(cmdf.sum(axis=1), axis=0)*100
    dims = (10, 10)
    fig, ax = plt.subplots(figsize=dims)
    sns.heatmap(percents, annot=True, cmap="coolwarm", center=0, vmin=0, vmax=100)
    ax.set_ylabel('Actual')    
    ax.set_xlabel('Predicted')
    cbar = ax.collections[0].colorbar
    cbar.set_ticks([0, 25, 50, 75, 100])
    cbar.set_ticklabels(['0%', '25%', '50%', '75%', '100%'])

In [4]:
hashtag_re = re.compile(r"#\w+")
mention_re = re.compile(r"@\w+")
url_re = re.compile(r"(?:https?://)?(?:[-\w]+\.)+[a-zA-Z]{2,9}[-\w/#~:;.?+=&%@~]*")

def preprocess(text):
    p_text = hashtag_re.sub("[hashtag]",text)
    p_text = mention_re.sub("[mention]",p_text)
    p_text = url_re.sub("[url]",p_text)
    p_text = ftfy.fix_text(p_text)
    return p_text.lower()

tokenise_re = re.compile(r"(\[[^\]]+\]|[-'\w]+|[^\s\w\[']+)") #([]|words|other non-space)
def tokenise(text):
    return tokenise_re.findall(text)

class Document:
    def __init__(self, meta={}):
        self.meta = meta
        self.tokens_fql = Counter() #empty Counter, ready to be added to with Counter.update.
        self.pos_fql = Counter()
        self.pos_list = [] #empty list for pos tags from running text.
        self.num_tokens = 0
        self.text = ""
        
    def extract_features_from_text(self, text):
        self.text += text
        p_text = preprocess(text)
        tokens = tokenise(p_text)
        self.num_tokens += len(tokens)
        self.tokens_fql.update(tokens) #updating Counter counts items in list, adding to existing Counter items.
        pos_tagged = nltk.pos_tag(tokens)
        pos = [tag[1] for tag in pos_tagged]
        self.pos_fql.update(pos)
        self.pos_list.extend(pos)
        
    def extract_features_from_texts(self, texts): #texts should be iterable text lines, e.g. read in from file.
        for text in texts:
            extract_features_from_text(text)

In [5]:
class DocumentProcessor(BaseEstimator, TransformerMixin):
    def __init__(self, process_method):
        self.process_method = process_method
    
    def fit(self, X, y=None): #no fitting necessary, although could use this to build a vocabulary for all documents, and then limit to set (e.g. top 1000).
        return self

    def transform(self, documents):
        for document in documents:
            yield self.process_method(document)

In [25]:
def get_tokens_fql(document):
    return document.tokens_fql

def get_pos_fql(document):
    return document.pos_fql

def read_list(file):
    with open(file) as f:
        items = []
        lines = f.readlines()
        for line in lines:
            items.append(line.strip())
    return items

fws = read_list("functionwords.txt")

def get_fws_fql(document):
    fws_fql = Counter({t: document.tokens_fql[t] for t in fws}) #dict comprehension, t: fql[t] is token: freq.
    return +fws_fql

def get_text_stats(document):
    ttr = len(document.tokens_fql) / document.num_tokens
    return {'avg_token_length': document.average_token_length(), 'ttr': ttr }


Here the Tweets are imported and put into a `Document` instance for each Tweet. This could be edited easily to use in CountVectorizer (as per week 17's lab), just return a list containing the tweet text, and a list containing the labels. All Tweets available from SemEval data are combined here, allowing for our own train/test split or cross-validation.

In [7]:
def import_tweets(file, label):
    metadata = {'label': label}
    with open(file) as f:
        tweets = f.readlines()
        for tweet in tweets:
            doc = Document(meta=metadata)
            doc.extract_features_from_text(tweet)
            yield doc

In [8]:
corpus = []
corpus.extend(import_tweets("sentiment/all/negative.txt", "negative"))
corpus.extend(import_tweets("sentiment/all/positive.txt", "positive"))
corpus.extend(import_tweets("sentiment/all/neutral.txt", "neutral"))

In [9]:
y = [d.meta['label'] for d in corpus]
X = corpus

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 0, stratify=y)
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))


model2 = Pipeline([
    ('text_union', FeatureUnion(
        transformer_list = [
            ('pos_features', Pipeline([
                ('pos_processor', DocumentProcessor(process_method = get_pos_fql)),
                ('pos_vectorizer', DictVectorizer()),
            ])),
            ('fws_features', Pipeline([
                ('fws_processor', DocumentProcessor(process_method = get_fws_fql)),
                ('fws_vectorizer', DictVectorizer()),
            ])),
            ('stats_features', Pipeline([
                ('stats_processor', DocumentProcessor(process_method = get_text_stats)),
                ('stats_vectorizer', DictVectorizer()),
            ])),
        ],
    )),
    ('clf', LogisticRegression(solver='liblinear', random_state=0)),
])


model2.fit(X_train, y_train)



predictions = model2.predict(X_test)

print("Accuracy: ", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

confusion_matrix_heatmap(confusion_matrix(y_test,predictions), ['M','F'])


model2.score(X_test,y_test)


17804 4452
17804 4452


AttributeError: 'Document' object has no attribute 'average_token_length'

## Task

Using code and features from previous labs, build and evaluate a sentiment classifier, which classifies individual Tweets into `positive`, `negative` or `neutral`.

Marks will be given as follows.
- 3 marks will be given for a well implemented and evaluated with at least 2 advanced feature sets (see below).
- 2 marks will be given for a completed, and evaluated, basic classifier utilising features from previous labs.
- 1 mark will be given for a full attempt.
This exercise must be demonstrated in Week 20's lab.

Advanced feature sets:
- Specific emojis
- Just adjectives
- Words from a sentiment lexicon (e.g. see "Opinion Lexicon" from https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html#lexicon).
- (very advanced) Word embeddings (ask Andrew in Week 19 lab)

In [15]:
X_train[0].tokens_fql


Counter({'video': 1,
         ':': 1,
         'endorsing': 1,
         '[hashtag]': 4,
         'with': 1,
         'the': 1,
         'kurt': 1,
         'cobain': 1,
         'going': 1,
         'into': 1,
         'a': 1,
         'minor': 1,
         '3rd': 1,
         'setting': 1,
         '(': 1,
         'at': 1,
         '...': 1,
         '[url]': 1})

In [23]:
tt = []
for doc in X:
    tt.append(get_tokens_fql(doc))
    
print(tt[0])

yy = tt[0]

    


Counter({'california': 1, 'may': 1, 'label': 1, "monsanto's": 1, 'roundup': 1, 'as': 1, "'known": 1, 'to': 1, 'cause': 1, "cancer'": 1, '|': 1, 'natural': 1, 'society': 1, '[url]': 1, 'well': 1, "it's": 1, 'about': 1, 'time': 1, '!': 1})
<bound method Counter.most_common of Counter({'california': 1, 'may': 1, 'label': 1, "monsanto's": 1, 'roundup': 1, 'as': 1, "'known": 1, 'to': 1, 'cause': 1, "cancer'": 1, '|': 1, 'natural': 1, 'society': 1, '[url]': 1, 'well': 1, "it's": 1, 'about': 1, 'time': 1, '!': 1})>
<bound method Counter.most_common of Counter({'i': 2, 'on': 2, 'why': 1, 'in': 1, 'the': 1, 'name': 1, 'of': 1, 'monsanto': 1, 'am': 1, 'sitting': 1, '@': 1, 'home': 1, 'saturday': 1, 'night': 1, 'trolling': 1, 'for': 1, 'used': 1, 'rvs': 1, 'craigslist': 1, '..': 1, 'oh': 1, 'yeah': 1, ',': 1, 'have': 1, 'no': 1, 'date': 1, '.': 1, '[hashtag]': 1})>
<bound method Counter.most_common of Counter({'.': 2, 'hey': 1, 'guys': 1, 'i': 1, 'got': 1, 'home': 1, 'from': 1, 'holiday': 1, 'in'

<bound method Counter.most_common of Counter({'my': 2, 'and': 2, 'ordered': 1, 'back': 1, 'pack': 1, 'off': 1, 'of': 1, 'amazon': 1, '3': 1, 'weeks': 1, 'ago': 1, 'it': 1, 'turns': 1, 'out': 1, 'they': 1, 'lost': 1, 'package': 1, 'school': 1, 'starts': 1, 'tomorrow': 1, '[hashtag]': 1})>
<bound method Counter.most_common of Counter({'this': 2, '[mention]': 1, 'how': 1, 'about': 1, 'you': 1, 'make': 1, 'a': 1, 'system': 1, 'that': 1, "doesn't": 1, 'eat': 1, 'my': 1, 'friggin': 1, 'discs': 1, '.': 1, 'is': 1, 'the': 1, '2nd': 1, 'time': 1, 'has': 1, 'happened': 1, 'and': 1, 'i': 1, 'am': 1, 'so': 1, 'sick': 1, 'of': 1, 'it': 1, '!': 1})>
<bound method Counter.most_common of Counter({'"': 2, 'prime': 2, 'arriving': 1, 'tuesday': 1, 'is': 1, 'decidedly': 1, 'the': 1, 'least': 1, 'my': 1, 'amazon': 1, 'shipping': 1, 'has': 1, 'ever': 1, 'been': 1, '.': 1, 'wha': 1, 'happen': 1, '?': 1})>
<bound method Counter.most_common of Counter({'[mention]': 2, ',': 2, 'a': 2, '[hashtag]': 1, '2nd': 1, 

<bound method Counter.most_common of Counter({'[hashtag]': 3, 'day': 3, 'will': 1, 'spend': 1, 'nye': 1, 'in': 1, 'utah': 1, 'vs': 1, ',': 1, 'play': 1, 'on': 1, 'mlk': 1, '(': 1, 'jan': 1, '.': 1, '18': 1, ')': 1, '..': 1, 'good': 1, 'news': 1, 'tho': 1, ':': 1, 'team': 1, 'is': 1, 'off': 1, "valentine's": 1, '&': 1, 'april': 1, "fool's": 1})>
<bound method Counter.most_common of Counter({'to': 2, 'go': 2, 'see': 2, 'and': 2, 'everyone': 1, 'went': 1, '[url]': 1, 'on': 1, 'monday': 1, "i'm": 1, 'out': 1, 'here': 1, 'about': 1, 'slipknot': 1, 'bullet': 1, 'for': 1, 'my': 1, 'valentine': 1, 'tomorrow': 1, '.': 1})>
<bound method Counter.most_common of Counter({'to': 2, '[hashtag]': 2, 'this': 1, 'man': 1, 'made': 1, 'up': 1, 'my': 1, 'mind': 1, 'oppose': 1, 'when': 1, 'he': 1, 'told': 1, 'tsipras': 1, 'hold': 1, 'his': 1, 'head': 1, 'high': 1, '[url]': 1})>
<bound method Counter.most_common of Counter({'[mention]': 2, 'for': 2, '.': 2, 'anyone': 1, 'selling': 1, 'tickets': 1, 'bullet': 

<bound method Counter.most_common of Counter({'going': 2, 'and': 2, 'roped': 2, 'into': 2, "i'm": 1, 'to': 1, 'see': 1, 'paper': 1, 'towns': 1, 'tomorrow': 1, 'with': 1, 'kelvin': 1, 'his': 1, 'friend': 1, 'who': 1, 'him': 1, 'he': 1, 'me': 1, 'it': 1, 'fun': 1})>
<bound method Counter.most_common of Counter({'room': 2, 'sat': 1, 'in': 1, 'my': 1, ',': 1, 'listening': 1, 'to': 1, 'the': 1, 'and': 1, 'frank': 1, 'ocean': 1, '....': 1, 'how': 1, 'moist': 1, 'am': 1, 'i': 1})>
<bound method Counter.most_common of Counter({'.': 3, 'hope': 1, 'you': 1, 'had': 1, 'a': 1, 'great': 1, 'wednesday': 1, 'time': 1, 'for': 1, 'me': 1, 'to': 1, 'watch': 1, 'big': 1, 'brother': 1, 'yes': 1, "i'm": 1, 'addicted': 1, '-': 1, 'i': 1, 'admit': 1, 'it': 1, 'denise': 1, '[hashtag]': 1, '[url]': 1})>
<bound method Counter.most_common of Counter({'.': 2, 'what': 1, 'a': 1, 'huge': 1, 'match': 1, 'tomorrow': 1, 'in': 1, 'the': 1, 'us': 1, 'amateur': 1, 'quarterfinals': 1, '--': 1, 'ncaa': 1, 'champ': 1, 'brys

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

